In [1]:
import pandas as pd
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
import matplotlib.pyplot as plt
import seaborn as sns
import os
from preprocessing import Feature_Extractor, data_preprocessing


In [2]:
symbol = "FPT"
predictest = pd.read_csv(f'{symbol}_LSTM_evaluate_result.csv')

In [3]:
predictest.dtypes

Unnamed: 0     int64
Date          object
predicted     object
dtype: object

In [4]:
def convert_to_float(x):
    return float(x[0])

In [5]:
if predictest['predicted'].apply(lambda x: isinstance(x, str)).any():
    predictest['predicted'] = predictest['predicted'].apply(eval).apply(convert_to_float)
predictest.dtypes

Unnamed: 0      int64
Date           object
predicted     float64
dtype: object

In [6]:
#Lưu file mới
predictest = predictest.drop(columns=['Unnamed: 0'], errors='ignore')
output_file_path = os.path.join(f'{symbol}_LSTM_evaluate_result.csv')
predictest.to_csv(output_file_path, index=False)

In [8]:
df = pd.read_csv(f'DataTrainLSTM/{symbol}.csv')
df.head


<bound method NDFrame.head of             Date     Close      Open      High       Low
0     2019-01-28   21948.0   22156.5   22156.5   21948.0
1     2019-01-29   22156.5   22104.4   22156.5   21895.8
2     2019-01-30   22521.4   22156.5   22625.7   22104.4
3     2019-01-31   22521.4   22573.6   22625.7   22365.0
4     2019-02-01   22208.6   22521.4   22521.4   21635.2
...          ...       ...       ...       ...       ...
1327  2024-05-27  132800.0  133000.0  133500.0  130600.0
1328  2024-05-28  137000.0  134000.0  137000.0  133100.0
1329  2024-05-29  135700.0  137100.0  138200.0  134800.0
1330  2024-05-30  135000.0  133800.0  135000.0  131900.0
1331  2024-05-31  134600.0  135000.0  136800.0  133800.0

[1332 rows x 5 columns]>

In [9]:
# Hợp nhất hai tệp CSV theo cột Date
df_merged = pd.merge(df, predictest, on='Date', how='left')
df_merged['Close'] = df_merged['predicted'].combine_first(df_merged['Close'])
df_merged.drop(columns=['predicted'], inplace=True)
cutoff_date = '2024-05-17'
df_filtered = df_merged[df_merged['Date'] <= cutoff_date]
output_folder = 'PredictTestData'
output_file_path = os.path.join(output_folder, f'{symbol}LSTMpredictions.csv')
df_filtered.to_csv(output_file_path, index=False)